### Calculate deltaPhi matrix from combinations

Steps are:
- Take $\phi$ and $\eta$ data arrays,starts and stops. 
- Get combinations indices in same manner as done earlier
- For each pair, calculate deltaR as $$\delta r = \sqrt{\delta \eta^{2} + \delta \phi^{2}}$$ where $$\delta \eta = \eta_{2} - \eta_{1} \\ \delta \phi = \phi_{2} - \phi_{1}$$
- Get the matrix $$\delta R = \{ \delta r_{ij}\} \quad \forall \: (i,j) \: \in \: \mathrm{combinations\_ indices}$$


In [1]:
import numpy
import pycuda.driver as cuda
from pycuda.compiler import *
import pycuda.autoinit

In [2]:
# Adapted from gen-reco script by Jim

NUMEVENTS = 100      # exact number of events
AVENUMJETS = 10      # average number of jets per event
PHILOW = -numpy.pi   # bounding box of phi (azimuthal angle) and eta (~polar angle)
PHIHIGH = numpy.pi
ETALOW = -5
ETAHIGH = 5
ERRPHI = 0.01        # detector resolution
ERRETA = 0.01
RECOPROB = 0.95      # probability of not reconstructing a real jet
AVENUMFAKES = 1      # average number of spurious (fake) recontstructions

# simulate the generator-level jets
numgenjets = numpy.random.poisson(AVENUMJETS, NUMEVENTS).astype(numpy.int32)
genstops = numpy.cumsum(numgenjets).astype(numpy.int32)
genstarts = numpy.empty_like(genstops)
genstarts[0] = 0
genstarts[1:] = genstops[:-1]
genphi = numpy.random.uniform(PHILOW, PHIHIGH, genstops[-1]).astype(numpy.float32)
geneta = numpy.random.uniform(ETALOW, ETAHIGH, genstops[-1]).astype(numpy.float32)

# simulate mismeasurement (error in reconstructing phi and eta)
phiwitherr = genphi + numpy.random.normal(0, ERRPHI, genstops[-1]).astype(numpy.float32)
etawitherr = geneta + numpy.random.normal(0, ERRETA, genstops[-1]).astype(numpy.float32)

In [3]:
mod = SourceModule('''
#include <cmath>
extern "C"{
__device__ float deltaeta(float eta1,float eta2)
{
    return eta1-eta2;
}

__device__ float deltaphi(float phi1, float phi2)
{
    return phi1-phi2;
}

__global__ void deltaR(float* deltar,float* eta1, float*eta2, float* phi1, float* phi2,int* length, int* start, int* stop, int* pairs_len,int* lengths)
{
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j = blockIdx.y*blockDim.y + threadIdx.y;
    int k = blockIdx.z*blockDim.z + threadIdx.z;
    if (i <length[0]){
        if(j<lengths[i] && k<lengths[i]){
            deltar[pairs_len[i] + j*lengths[i] + k] = hypotf(deltaeta(eta1[j+start[i]], eta2[k+start[i]]), deltaphi(phi1[j + start[i]], phi2[k+start[i]]));
        }
    }
}
}
''', no_extern_c = True)

In [4]:
lengths = genstops-genstarts

In [5]:
pairs_len = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_len[1:] = numpy.cumsum(lengths*lengths)

In [6]:
lengths.dtype

dtype('int32')

In [7]:
pairs_len[:5]

array([  0,  81, 225, 306, 406])

In [8]:
deltar = numpy.empty(pairs_len[-1], dtype=numpy.float32)

In [9]:
base_len = numpy.array(len(lengths)).astype(numpy.int32)

In [10]:
func = mod.get_function("deltaR")

In [11]:
genstarts

array([  0,   9,  21,  30,  40,  43,  49,  63,  70,  80,  91, 111, 126,
       143, 148, 159, 169, 183, 190, 204, 215, 227, 238, 243, 256, 263,
       279, 285, 299, 304, 312, 318, 329, 339, 350, 365, 378, 388, 393,
       407, 417, 426, 432, 438, 448, 459, 471, 479, 490, 499, 517, 525,
       538, 546, 551, 558, 572, 582, 595, 605, 612, 621, 631, 642, 649,
       659, 673, 683, 694, 705, 713, 726, 738, 754, 764, 771, 785, 790,
       798, 805, 813, 818, 826, 832, 849, 856, 862, 867, 881, 888, 898,
       905, 921, 933, 938, 949, 966, 975, 986, 994])

In [12]:
func(cuda.InOut(deltar),cuda.In(geneta), cuda.In(etawitherr), cuda.In(genphi), cuda.In(phiwitherr), 
    cuda.In(base_len),cuda.In(genstarts), cuda.In(genstops), cuda.In(pairs_len), cuda.In(lengths), block=(1,8,8),grid = (len(numgenjets), 20, 20))

In [15]:
for i in range(6):
    print("Event:{} \n {}\n".format(i, deltar[pairs_len[i]:pairs_len[i+1]]))

Event:0 
 [  1.14196595e-02   5.19899750e+00   7.04922342e+00   1.29171610e+00
   2.39712572e+00   5.07040691e+00   4.51480687e-01   7.31054735e+00
   7.68462753e+00   5.20713377e+00   2.53946334e-03   2.24332309e+00
   3.95715451e+00   4.05498648e+00   1.37319100e+00   5.30775023e+00
   2.61706972e+00   2.91166520e+00   7.06925964e+00   2.24845791e+00
   2.00159010e-02   5.92645502e+00   6.26172018e+00   3.45103931e+00
   7.07439947e+00   3.28391242e+00   6.65451705e-01   1.29740429e+00
   3.96610832e+00   5.92615414e+00   1.88953765e-02   1.52069354e+00
   3.78251481e+00   1.54061902e+00   6.02253580e+00   6.58010340e+00
   2.41428518e+00   4.05765295e+00   6.25537920e+00   1.54250383e+00
   6.85566710e-03   3.35291362e+00   2.80489111e+00   5.50820589e+00
   6.92938900e+00   5.08530521e+00   1.38405097e+00   3.45555902e+00
   3.78481531e+00   3.35452056e+00   1.00181829e-02   5.29501677e+00
   2.23713326e+00   4.08760977e+00   4.54767257e-01   5.31467438e+00
   7.06399775e+00   1.55

11247